In [1]:
#MIT License

In [2]:
import operator
import collections

In [12]:
class Poly(tuple):
    def __new__(cls, obj=None):
        if not obj:
            return super().__new__(cls)
        elif isinstance(obj, collections.abc.Mapping):
            if not obj:
                return cls()
            return cls(obj.get(i, 0) for i in range(max(obj), -1, -1))
        
        obj = list(obj)
        while obj[0] == 0:
            obj.pop(0)
            if not obj:
                return cls()
        return super().__new__(cls, obj)
    
    def todict(self):
        return {e: c for e, c in enumerate(reversed(self)) if c}
    
    def __add__(self, other):
        together = collections.Counter(self.todict()) + collections.Counter(other.todict())
        
        cls = type(self)
        return cls(together)
    
    
    def __mul__(self, other):
        total = [0] * (len(self) + len(other) - 1)
        for i, val1 in self.todict().items():
            for j, val2 in other.todict().items():
                total[i + j] += val1 * val2
                
        cls = type(self)
        return cls(reversed(total))
            
    def __str__(self):
        """ This is a mess """
        l = {e: ('{:+}'.format(c)[0] + ' ', abs(c)) for e, c in self.todict().items()}
        if not l:
            return ''

        largest = l[max(l)]
        largest = largest[0][0] if largest[0][0] is '-' else '', largest[1]
        l[max(l)] = largest

        return ' '.join('{}{}{}{}'.format(
            c[0],
            c[1],
            'x' if e > 0 else '',
            ('^' + str(e)) if e > 1 else '')
                       for e, c in sorted(
                           l.items(),
                           key=operator.itemgetter(0),
                           reverse=True)
                      )
    
    def __repr__(self):
        return '{}({})'.format(type(self).__name__, super().__repr__())

In [13]:
import unittest
import hypothesis
import hypothesis.strategies as st

In [14]:
liststrat = st.lists(st.integers() | st.floats())
dictstrat = st.dictionaries(st.integers(min_value=0, max_value=20), st.integers() | st.floats(), max_size=10)
class PolyCase(unittest.TestCase):
    def testAcceptsNone(self):
        try:
            Poly()
        except Exception as e:
            self.fail(e)
    @hypothesis.given(liststrat)
    def testAcceptsList(self, l):
        try:
            Poly(l)
        except Exception as e:
            self.fail("Poly() raised {} unexpectedly!".format(e))
    
    @hypothesis.given(dictstrat)
    def testAcceptsDict(self, d):
        try:
            d, Poly(d)
        except Exception as e:
            print(e)
        
    def testPretty(self):
        pretty_tests = {
            (4, 3, 2): '4x^2 + 3x + 2',
            (2, 8, 3): '2x^2 + 8x + 3',
            (8, 6, 7, 9): '8x^3 + 6x^2 + 7x + 9',
            (7, -3, 5, -6): '7x^3 - 3x^2 + 5x - 6',
            (5, 0, 2): '5x^2 + 2',
            (-7, 0, 3, 5, 0, -2): '-7x^5 + 3x^3 + 5x^2 - 2',
            (-7, 0, 0, 5, 0, 0): '-7x^5 + 5x^2',
        }
        for test, string in pretty_tests.items():
            self.assertEqual(str(Poly(test)), string)
            
    def testEquality(self):
        self.assertEqual(Poly(), Poly())
        self.assertEqual(Poly((3, 4, 5)), Poly((3, 4, 5)))
        self.assertNotEqual(Poly((5, 4, 3)), Poly((3, 4, 5)))
        self.assertEqual(Poly((7, 8, 3, 2, 9)), Poly((7, 8, 3, 2, 9)))
        
    def testAdd(self):
        self.assertEqual(Poly((3,)) + Poly((2,)), Poly((5,)))
        self.assertEqual(Poly((3, 1, 2)) + Poly((1, 2, 3)), Poly((4, 3, 5)))
        self.assertEqual(Poly((15, 16, 19, 18)) + Poly((11, 19)), Poly((15, 16, 30, 37)))
    
    def testMul(self):
        self.assertEqual(Poly((4, -5)) * Poly((2, 3, -6)), Poly((8, 2, -39, 30)))
        self.assertEqual(Poly((3, 2)) * Poly((4, -7, 5)), Poly((12, -13, 1, 10)))
    
    @hypothesis.given(dictstrat | liststrat)
    def testDict(self, o1):
        p1 = Poly(o1)
        p2 = Poly(p1.todict())
        self.assertEqual(p1, p2)
        
unittest.main(argv=['first-arg-is-ignored'], exit=False,)

'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribut

.

'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribut

..EE...

Falsifying example: testDict(self=<__main__.PolyCase testMethod=testDict>, o1=[5e-324])
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attr

.


'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribute 'get'
'generator' object has no attribu

.EE...

Falsifying example: testDict(self=<__main__.TestPoly testMethod=testDict>, o1=[1])



ERROR: testAdd (__main__.PolyCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-a1f53b82bb82>", line 43, in testAdd
    self.assertEqual(Poly((3,)) + Poly((2,)), Poly((5,)))
  File "<ipython-input-12-d690447e265e>", line 25, in __add__
    return cls(together)
  File "<ipython-input-12-d690447e265e>", line 11, in __new__
    obj = list(obj)
  File "<ipython-input-12-d690447e265e>", line 8, in <genexpr>
    z = (obj.get(i, 0) for i in range(max(obj), -1, -1))
AttributeError: 'generator' object has no attribute 'get'

ERROR: testDict (__main__.PolyCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-a1f53b82bb82>", line 52, in testDict
    def testDict(self, o1):
  File "/usr/local/lib/python3.4/dist-packages/hypothesis/core.py", line 646, in wrapped_test
    state.run()
  File "/usr/local/lib/python3.4/dist-packages/